# import packages

In [ ]:
# setting base

import os

import warnings, sys, os ## system
if not sys.warnoptions:
    warnings.simplefilter("ignore") # ignore warnings
    
import mne
from mne.decoding import (SlidingEstimator, GeneralizingEstimator, Scaler,
                          cross_val_multiscore, Vectorizer)

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import scipy.io

# load files

## load EEG file

In [ ]:
# load file

user_path = os.path.expanduser('~')
data_path = os.path.join('_')
# remove _, and enter your data path instead
# enter the path to data folder not file itself
vhdr_path = os.path.join('_')
# remove _, and enter your data path instead
# the file is named as 'sub-02_decoder-distractor'
# include data_path also
# !caution! you should load 'vhdr' file

raw = mne.io.read_raw_brainvision(vhdr_path,eog=('HEOG','HEOG1','VEOG'))
# data used in this project is collected by 'brainvision'

In [ ]:
# apply pass filter

raw.load_data().filter(0.5, 40) 
# import filter (high-pass, low-pass)

events = mne._(raw) 
# remove _, and enter the right function for defining triggers
# how can you conduct this step?

In [ ]:
# define triggers

total_target_id = {'first_target_object': 111, 'first_target_scene': 112,
                   'second_target_object': 211, 'second_target_scene': 212,
                   'first_dummy_dist': 120, 'second_dummy_dist': 220,
                   'first_dist_object': 121, 'first_dist_scene': 122,
                   'second_dist_object': 221, 'second_dist_scene': 222,
                   'cue_object': 31, 'cue_scene': 32,
                   'probe_object': 41, 'probe_scene': 42}

fir_tar_id = {'first_target_object': 111, 'first_target_scene': 112}
sec_tar_id = {'second_target_object': 211, 'second_target_scene': 212}
cue_id = {'cue_object': 31, 'cue_scene': 32}

In [ ]:
# re-reference signal

raw.set_eeg_reference('average', projection=True)
raw.plot()

## load trial information

In [ ]:
# load task design

nd_dsgn_path = os.path.join('_',
                            'sub-02_decode_behavior_without-dist_dsgn.mat')
d_dsgn_path = os.path.join('_',
                           'sub-02_decode_behavior_with-dist_dsgn.mat')
# remove _, and enter your data path
# include data_path also

nd_dsgn = scipy.io.loadmat(nd_dsgn_path)
d_dsgn = scipy.io.loadmat(d_dsgn_path)
## design matrix
# c0: stimulus sequence - 1:object|scene / 2:scene|obejct
# c1: distractor - 1 X | 2 O -> tasktype
# c2: target category - 1:object / 2:scene
# c3: WM test - 1 same | 2 different

print(nd_dsgn)
print(d_dsgn)

In [ ]:
# load behavior data

nd_resp_path = os.path.join('_',
                            'sub-02_decode_behavior_without-dist_wmr.mat')
d_resp_path = os.path.join('_',
                           'sub-02_decode_behavior_with-dist_wmr.mat')
# remove _, and enter your data path
# include data_path also

nd_resp = scipy.io.loadmat(nd_resp_path)
d_resp = scipy.io.loadmat(d_resp_path)
## response matrix
# c0: response - 1 same | 2 different
# c1: reaction time

print(nd_resp)
print(d_resp)

In [ ]:
# make informations as data frame

# integrate experimental data
dum_2 = np.zeros((200,1))
dum_4 = np.zeros((400,1))

tot_nd = np.hstack((nd_dsgn['dsgn'], dum_2, nd_resp['WM_R']))
tot_d = np.hstack((d_dsgn['dsgn'], dum_4, d_resp['WM_R']))

# organize data frame using pandas
idx = ['sequence', 'dist', 'target cat', 'wm_test', 'dum', 'resp', 'RT']

tot_nd = pd.DataFrame(tot_nd, columns=idx)
tot_d = pd.DataFrame(tot_d, columns=idx)

print(tot_nd)
print(tot_d)

# preprocess data

## remove blink signal using SSP

In [ ]:
# make SSP projector

blink_proj = mne.preprocessing.compute_proj_eog(raw, tmax=4.0, n_mag=0,
                                                ch_name=['HEOG','HEOG1','VEOG'])

In [ ]:
# add SSP projector

raw.add_proj(blink_proj[0])

## remove blinked trial by rejecting trials

### keep the result information in trial information later
**include below sentences when you make dataframe** <br>
tot_np = np.vstack((ne_np, e_np))<br>
rej_t = np.zeros((len(tot_np),1))<br>
rej_t[rej_trial] = 1

In [ ]:
# duplicate data
rej_trial = raw.copy()

# find eye blink
eog_events = mne.preprocessing.find_eog_events(rej_trial) 

# find blink signal
onsets = eog_events[:, 0] / rej_trial.info['sfreq'] - 0.25
durations = [0.5] * len(eog_events)
descriptions = ['bad blink'] * len(eog_events)
blink_annot = mne.Annotations(onsets, durations, descriptions,
                              orig_time=rej_trial.info['meas_date'])
rej_trial.set_annotations(blink_annot)

In [ ]:
# make epochs and check blinked trials

# define time interval
tmin, tmax = -0.200, 3.600 # cut time-course

# make epochs based on the presence of first target
temp_epochs = mne.Epochs(rej_trial, events[0], fir_tar_id, tmin, tmax, 
                         proj=True, picks=ch_name, baseline=(-0.2, 0.0),
                         preload=True, reject=None, decim=3, verbose='error')

In [ ]:
# select trials to be removed away

Seq = np.array(range(600))
rej_trial = np.delete(Seq, temp_epochs.events[:,1])

## epoching

In [ ]:
# define time interval and channel name

tmin, tmax = -0.200, 0.600 # cut time-course
ch_name = ['Fz', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1',
           'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'CP6',
           'CP2', 'Cz', 'C4', 'T8', 'POz', 'FC6', 'FC2', 'F4', 'F8',]

In [ ]:
# make target epoch

fir_tar_epochs = mne.Epochs(raw, events[0], _, tmin, tmax, 
                            proj=True, picks=ch_name, baseline=(-0.2, 0.0),
                            preload=True, reject=None, decim=3, verbose='error')

sec_tar_epochs = mne.Epochs(raw, events[0], _, tmin, tmax,
                            proj=True, picks=ch_name, baseline=(-0.2, 0.0),
                            preload=True, reject=None, decim=3, verbose='error')
# remove _s, and enter right triggers for each epochs

In [ ]:
# make cue epoch

cue_epochs = mne.Epochs(raw, events[0], _, tmin, tmax, 
                        proj=True, picks=ch_name, baseline=(-0.2, 0.0),
                        preload=True, reject=None, decim=3, verbose='error')
# remove _, and enter right trigger for epochs

# define trial infromation

## define the stimuli condition of trials

In [ ]:
# no-expectation block

# the sequence of trials
ne_base = np.array(range(len(tot_nd['resp'])))[:,None]

# the category of at that sequence
# 1: object | 2: scene
ne_seq = np.zeros((len(ne_base),2))
seq_OS = tot_nd[(tot_nd['sequence'] == 1)].index.values
seq_SO = tot_nd[(tot_nd['sequence'] == 2)].index.values
ne_seq[seq_OS,:] = [1, 2]
ne_seq[seq_SO,:] = [2, 1]

# the cued category of that trial
# 1: object | 2: scene
ne_cue = np.zeros((len(ne_base),1))
cue_O = tot_nd[(tot_nd['target cat'] == 1)].index.values
cue_S = tot_nd[(tot_nd['target cat'] == 2)].index.values
ne_cue[cue_O] = 1
ne_cue[cue_S] = 2

# the cued sequence of that trial
# 1: first | 2: second
ne_po = np.zeros((len(ne_base),1))
for i in range(len(ne_po)):
    if (nd_dsgn['dsgn'][i,0] == 1):
        if (nd_dsgn['dsgn'][i,2] == 1):
            ne_po[i] = 1
        elif (nd_dsgn['dsgn'][i,2] == 2):
            ne_po[i] = 2
    elif (nd_dsgn['dsgn'][i,0] == 2):
        if (nd_dsgn['dsgn'][i,2] == 1):
            ne_po[i] = 2
        elif (nd_dsgn['dsgn'][i,2] == 2):
            ne_po[i] = 1

# check whether the subject was right or wrong
# 0: wrong ansewer | 1: right ansewer
ne_ans = np.zeros((len(ne_base),1))
ans_C = tot_nd[(tot_nd['wm_test'] == tot_nd['resp'])].index.values
ne_ans[ans_C] = 1

# response time of that trial
ne_rt = np.array(tot_nd['RT'])[:,None]

# check whether there were distractors or not
# in not-expectation block, there was no distractor in every trials for sure
dum = np.zeros((len(ne_base),1))
            
# check whether the subject ansewered in that trial
# 0: answered | 1: not answered
ne_nan = np.zeros((len(ne_base),1))
nan_t = tot_nd[np.isnan(tot_nd['resp'])].index.values
ne_nan[nan_t] = 1

ne_np = np.hstack((ne_base, ne_seq, ne_cue, ne_po, ne_ans, ne_rt, dum, ne_nan))

In [ ]:
# expectation block

# the sequence of trials
e_base = np.array(range(len(tot_d['resp'])))[:,None] + len(tot_nd['resp'])

e_seq = np.zeros((len(e_base),2))
seq_OS = tot_d[(tot_d['sequence'] == 1)].index.values
seq_SO = tot_d[(tot_d['sequence'] == 2)].index.values
e_seq[seq_OS,:] = [1, 2]
e_seq[seq_SO,:] = [2, 1]

# the category of at that sequence
# 1: object | 2: scene
e_cue = np.zeros((len(e_base),1))
cue_O = tot_d[(tot_d['target cat'] == 1)].index.values
cue_S = tot_d[(tot_d['target cat'] == 2)].index.values
e_cue[cue_O] = 1
e_cue[cue_S] = 2

# the cued sequence of that trial
# 1: first | 2: second
e_po = np.zeros((len(e_base),1))
for i in range(len(e_po)):
    if (d_dsgn['dsgn'][i,0] == 1):
        if (d_dsgn['dsgn'][i,2] == 1):
            e_po[i] = 1
        elif (d_dsgn['dsgn'][i,2] == 2):
            e_po[i] = 2
    elif (d_dsgn['dsgn'][i,0] == 2):
        if (d_dsgn['dsgn'][i,2] == 1):
            e_po[i] = 2
        elif (d_dsgn['dsgn'][i,2] == 2):
            e_po[i] = 1
            
# check whether the subject was right or wrong
# 0: wrong ansewer | 1: right ansewer
e_ans = np.zeros((len(e_base),1))
ans_C = tot_d[(tot_d['wm_test'] == tot_d['resp'])].index.values
e_ans[ans_C] = 1

# response time of that trial
e_rt = np.array(tot_d['RT'])[:,None]

# check whether there were distractors or not
# 1: not-distracted | 2: distracted
e_dist = np.zeros((len(e_base),1))
nd_pos = tot_d[(tot_d['dist'] == 1)].index.values
d_pos = tot_d[(tot_d['dist'] == 2)].index.values
e_dist[nd_pos] = 1
e_dist[d_pos] = 2

# check whether the subject ansewered in that trial
# 0: answered | 1: not answered
e_nan = np.zeros((len(e_base),1))
nan_t = tot_d[np.isnan(tot_d['resp'])].index.values
e_nan[nan_t] = 1

e_np = np.hstack((e_base, e_seq, e_cue, e_po, e_ans, e_rt, e_dist, e_nan))

In [ ]:
# integrate all the information into one data frame

tot_np = np.vstack((ne_np, e_np))
tot_df = pd.DataFrame(tot_np, columns=['posit', 'first_t', 'second_t', 'cue_C', 'cue_S',
                                       'corr', 'rt', 'dist', 'nan'])

print(tot_df)

## separate epochs according to conditions

### basic separation

In [ ]:
# I will separate epochs according to the condition about distractor

# separate according to the expectation about distractor
ne_trials = tot_df[(tot_df['posit'] < 200) & (tot_df['nan'] == 0)]['posit']
e_trials = tot_df[(tot_df['posit'] > 199) & (tot_df['nan'] == 0)]['posit']

# separate according to the presence of distractor
e_nd_trials = tot_df[(tot_df['posit'] > 199) & (tot_df['nan'] == 0)
                     & (tot_df['dist'] == 1)]['posit']
e_d_trials = tot_df[(tot_df['posit'] > 199) & (tot_df['nan'] == 0)
                    & (tot_df['dist'] == 2)]['posit']

In [ ]:
# translate into int array

ne_trials = np.array(ne_trials).astype(int)
e_trials = np.array(e_trials).astype(int)

e_nd_trials = np.array(e_nd_trials).astype(int)
e_d_trials = np.array(e_d_trials).astype(int)

### example separation

In [ ]:
# I will separate epochs according to whether the subject answered right or wrong

# no-expectation block
ne_cor_trials = tot_df[(tot_df['posit'] < 200) & (tot_df['nan'] == 0)
                       & (tot_df['corr'] == 1)]['posit']
ne_wro_trials = tot_df[(tot_df['posit'] < 200) & (tot_df['nan'] == 0)
                       & (tot_df['corr'] == 0)]['posit']

# expectatin block (total)
e_cor_trials = tot_df[(tot_df['posit'] > 199) & (tot_df['nan'] == 0)
                      & (tot_df['corr'] == 1)]['posit']
e_wro_trials = tot_df[(tot_df['posit'] > 199) & (tot_df['nan'] == 0)
                      & (tot_df['corr'] == 0)]['posit']

# expectation block (distractor-absent)
e_nd_cor_trials = tot_df[(tot_df['posit'] > 199) & (tot_df['nan'] == 0)
                         & (tot_df['dist'] == 1) & (tot_df['corr'] == 1)]['posit']
e_nd_wro_trials = tot_df[(tot_df['posit'] > 199) & (tot_df['nan'] == 0)
                         & (tot_df['dist'] == 1) & (tot_df['corr'] == 0)]['posit']

# expectation block (distractor-present)
e_d_cor_trials = tot_df[(tot_df['posit'] > 199) & (tot_df['nan'] == 0)
                        & (tot_df['dist'] == 2) & (tot_df['corr'] == 1)]['posit']
e_d_wro_trials = tot_df[(tot_df['posit'] > 199) & (tot_df['nan'] == 0)
                        & (tot_df['dist'] == 2) & (tot_df['corr'] == 2)]['posit']

In [ ]:
# translate into int array

ne_cor_trials = np.array(ne_cor_trials).astype(int)
ne_wro_trials = np.array(ne_wro_trials).astype(int)

e_cor_trials = np.array(e_cor_trials).astype(int)
e_wro_trials = np.array(e_wro_trials).astype(int)

e_nd_cor_trials = np.array(e_nd_cor_trials).astype(int)
e_nd_wro_trials = np.array(e_nd_wro_trials).astype(int)

e_d_cor_trials = np.array(e_d_cor_trials).astype(int)
e_d_wro_trials = np.array(e_d_wro_trials).astype(int)

### make your own condition separation

In [ ]:
# I will separate epochs according to -

# no-expectation block


# expectatin block (total)


# expectation block (distractor-absent)


# expectation block (distractor-present)


In [ ]:
# translate into int array



### separate epochs according to condition (basic separation)

In [ ]:
# separate epochs according to expectation and distraction

# no-expectation
NE_FT_trial = fir_tar_epochs[ne_trials]
NE_ST_trial = sec_tar_epochs[ne_trials]
NE_C_trial = cue_epochs[ne_trials]

# expectation
E_FT_trial = fir_tar_epochs[e_trials]
E_ST_trial = sec_tar_epochs[e_trials]
E_C_trial = cue_epochs[e_trials]

# distractor-absent
E_ND_FT_trial = fir_tar_epochs[e_nd_trials]
E_ND_ST_trial = sec_tar_epochs[e_nd_trials]
E_ND_C_trial = cue_epochs[e_nd_trials]

# distractor-present
E_D_FT_trial = fir_tar_epochs[e_d_trials]
E_D_ST_trial = sec_tar_epochs[e_d_trials]
E_D_C_trial = cue_epochs[e_d_trials]

# conduct decoding

## organize data

### no-expectation block

In [ ]:
# no-expectation + first target

ne_FT_X = NE_FT_trial.get_data()  # n_epochs, n_eeg_channels, n_times
ne_FT_y = tot_df.iloc[ne_trials]['first_t']  # target event

In [ ]:
# no-expectation + first target

ne_ST_X = NE_ST_trial.get_data()  # n_epochs, n_eeg_channels, n_times
ne_ST_y = tot_df.iloc[ne_trials]['second_t']  # target event

In [ ]:
# no-expectation + first target

ne_C_X = NE_C_trial.get_data()  # n_epochs, n_eeg_channels, n_times
ne_C_y = tot_df.iloc[ne_trials]['cue_C']  # target event

### expectation block

In [ ]:
# no-expectation + first target

e_FT_X = E_FT_trial.get_data()  # n_epochs, n_eeg_channels, n_times
e_FT_y = tot_df.iloc[e_trials]['first_t']  # target event

In [ ]:
# no-expectation + first target

e_ST_X = E_ST_trial.get_data()  # n_epochs, n_eeg_channels, n_times
e_ST_y = tot_df.iloc[e_trials]['second_t']  # target event

In [ ]:
# no-expectation + first target

e_C_X = E_C_trial.get_data()  # n_epochs, n_eeg_channels, n_times
e_C_y = tot_df.iloc[e_trials]['cue_C']  # target event

### expectation block + distractor absent

In [ ]:
# no-expectation + first target

e_nd_FT_X = E_ND_FT_trial.get_data()  # n_epochs, n_eeg_channels, n_times
e_nd_FT_y = tot_df.iloc[e_nd_trials]['first_t']  # target event

In [ ]:
# no-expectation + first target

e_nd_ST_X = E_ND_ST_trial.get_data()  # n_epochs, n_eeg_channels, n_times
e_nd_ST_y = tot_df.iloc[e_nd_trials]['second_t']  # target event

In [ ]:
# no-expectation + first target

e_nd_C_X = E_ND_C_trial.get_data()  # n_epochs, n_eeg_channels, n_times
e_nd_C_y = tot_df.iloc[e_nd_trials]['cue_C']  # target event

### expectation block + distractor present

In [ ]:
# no-expectation + first target

e_d_FT_X = E_D_FT_trial.get_data()  # n_epochs, n_eeg_channels, n_times
e_d_FT_y = tot_df.iloc[e_d_trials]['first_t']  # target event

In [ ]:
# no-expectation + first target

e_d_ST_X = E_D_ST_trial.get_data()  # n_epochs, n_eeg_channels, n_times
e_d_ST_y = tot_df.iloc[e_d_trials]['second_t']  # target event

In [ ]:
# no-expectation + first target

e_d_C_X = E_D_C_trial.get_data()  # n_epochs, n_eeg_channels, n_times
e_d_C_y = tot_df.iloc[e_d_trials]['cue_C']  # target event

## cross-validation (temporal specific)

### no-expectation block

In [ ]:
# train classifier on first target in no-expectation block

# make pipeline
clf_pipe = make_pipeline(Scaler(NE_FT_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# organize decoder
time_decod = SlidingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
ne_FT_scores = cross_val_multiscore(time_decod, ne_FT_X, ne_FT_y, cv=None, n_jobs=None)

# mean scores of decoding according to time point
ne_FT_score = np.mean(ne_FT_scores, axis=0)

In [ ]:
# Plot the performance of decoder

fig, ax = plt.subplots()
ax.plot(NE_FT_trial.times, ne_FT_score, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('decoding performance : no-expectation block + first target')

In [ ]:
# train classifier on second target in no-expectation block

# make pipeline
clf_pipe = make_pipeline(Scaler(NE_ST_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# organize decoder
time_decod = SlidingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
ne_ST_scores = cross_val_multiscore(time_decod, ne_ST_X, ne_ST_y, cv=None, n_jobs=None)

# mean scores of decoding according to time point
ne_ST_score = np.mean(ne_ST_scores, axis=0)

In [ ]:
# Plot the performance of decoder

fig, ax = plt.subplots()
ax.plot(NE_ST_trial.times, ne_ST_score, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('decoding performance : no-expectation block + second target')

In [ ]:
# train classifier on cue in no-expectation block

# make pipeline
clf_pipe = make_pipeline(Scaler(NE_C_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# organize decoder
time_decod = SlidingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
ne_C_scores = cross_val_multiscore(time_decod, ne_C_X, ne_C_y, cv=None, n_jobs=None)

# mean scores of decoding according to time point
ne_C_score = np.mean(ne_C_scores, axis=0)

In [ ]:
# Plot the performance of decoder

fig, ax = plt.subplots()
ax.plot(NE_C_trial.times, ne_C_score, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('decoding performance : no-expectation block + cue')

### expectation block

In [ ]:
# train classifier on first target in expectation block

# make pipeline
clf_pipe = make_pipeline(Scaler(E_FT_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# organize decoder
time_decod = SlidingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_FT_scores = cross_val_multiscore(time_decod, _, _, cv=None, n_jobs=None)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_FT_score = np.mean(e_FT_scores, axis=0)

In [ ]:
# Plot the performance of decoder

fig, ax = plt.subplots()
ax.plot(E_FT_trial.times, e_FT_score, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('decoding performance : expectation block + first target')

In [ ]:
# train classifier on second target in expectation block

# make pipeline
clf_pipe = make_pipeline(Scaler(E_ST_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# organize decoder
time_decod = SlidingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_ST_scores = cross_val_multiscore(time_decod, _, _, cv=None, n_jobs=None)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_ST_score = np.mean(e_ST_scores, axis=0)

In [ ]:
# Plot the performance of decoder

fig, ax = plt.subplots()
ax.plot(E_ST_trial.times, e_ST_score, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('decoding performance : expectation block + second target')

In [ ]:
# train classifier on cue in expectation block

# make pipeline
clf_pipe = make_pipeline(Scaler(E_C_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# organize decoder
time_decod = SlidingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_C_scores = cross_val_multiscore(time_decod, _, _, cv=None, n_jobs=None)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_C_score = np.mean(e_C_scores, axis=0)

In [ ]:
# Plot the performance of decoder

fig, ax = plt.subplots()
ax.plot(E_C_trial.times, e_C_score, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('decoding performance : expectation block + cue')

### expectation block + distractor-absent

In [ ]:
# train classifier on first target in distractor-absent trials

# make pipeline
clf_pipe = make_pipeline(Scaler(E_ND_FT_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# organize decoder
time_decod = SlidingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_nd_FT_scores = cross_val_multiscore(time_decod, _, _, cv=None, n_jobs=None)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_nd_FT_score = np.mean(e_nd_FT_scores, axis=0)

In [ ]:
# Plot the performance of decoder

fig, ax = plt.subplots()
ax.plot(E_ND_FT_trial.times, e_nd_FT_score, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('decoding performance : distractor-absent trial + first target')

In [ ]:
# train classifier on second target in distractor-absent trials

# make pipeline
clf_pipe = make_pipeline(Scaler(E_ND_ST_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# organize decoder
time_decod = SlidingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_nd_ST_scores = cross_val_multiscore(time_decod, _, _, cv=None, n_jobs=None)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_nd_ST_score = np.mean(e_nd_ST_scores, axis=0)

In [ ]:
# Plot the performance of decoder

fig, ax = plt.subplots()
ax.plot(E_ND_ST_trial.times, e_nd_ST_score, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('decoding performance : distractor-absent trial + second target')

In [ ]:
# train classifier on cue in distractor-absent trials

# make pipeline
clf_pipe = make_pipeline(Scaler(E_ND_C_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# organize decoder
time_decod = SlidingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_nd_C_scores = cross_val_multiscore(time_decod, _, _, cv=None, n_jobs=None)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_nd_C_score = np.mean(e_nd_C_scores, axis=0)

In [ ]:
# Plot the performance of decoder

fig, ax = plt.subplots()
ax.plot(E_ND_C_trial.times, e_nd_C_score, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('decoding performance : distractor-absent trial + cue')

### expectation block + distractor-present

In [ ]:
# train classifier on first target in distractor-present trials

# make pipeline
clf_pipe = make_pipeline(Scaler(E_D_FT_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# organize decoder
time_decod = SlidingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_d_FT_scores = cross_val_multiscore(time_decod, _, _, cv=None, n_jobs=None)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_d_FT_score = np.mean(e_d_FT_scores, axis=0)

In [ ]:
# Plot the performance of decoder

fig, ax = plt.subplots()
ax.plot(E_D_FT_trial.times, e_d_FT_score, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('decoding performance : distractor-present trial + first target')

In [ ]:
# train classifier on second target in distractor-present trials

# make pipeline
clf_pipe = make_pipeline(Scaler(E_D_ST_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# organize decoder
time_decod = SlidingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_d_ST_scores = cross_val_multiscore(time_decod, _, _, cv=None, n_jobs=None)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_d_ST_score = np.mean(e_d_ST_scores, axis=0)

In [ ]:
# Plot the performance of decoder

fig, ax = plt.subplots()
ax.plot(E_D_ST_trial.times, e_d_ST_score, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('decoding performance : distractor-present trial + second target')

In [ ]:
# train classifier on cue in distractor-present trials

# make pipeline
clf_pipe = make_pipeline(Scaler(E_D_C_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# organize decoder
time_decod = SlidingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_d_C_scores = cross_val_multiscore(time_decod, _, _, cv=None, n_jobs=None)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_d_C_score = np.mean(e_d_C_scores, axis=0)

In [ ]:
# Plot the performance of decoder

fig, ax = plt.subplots()
ax.plot(E_D_C_trial.times, e_d_C_score, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('decoding performance : distractor-present trial + cue')

## cross-validation (temporal generalization)

### no-expectation block

In [ ]:
# train classifier on first target in no-expectation block

# make pipeline
clf_pipe = make_pipeline(Scaler(NE_FT_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# define the temporal generalization object
time_gen = GeneralizingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
ne_FT_tgs = cross_val_multiscore(time_gen, ne_FT_X, ne_FT_y, cv=None, n_jobs=5)

# mean scores of decoding according to time point
ne_FT_tg = np.mean(ne_FT_tgs, axis=0)

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(ne_FT_tg, interpolation='lanczos', origin='lower',
               cmap='RdBu_r', extent=NE_FT_trial.times[[0, -1, 0, -1]],
               vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding performance : no-expectation block + first target')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
# train classifier on first target in no-expectation block

# make pipeline
clf_pipe = make_pipeline(Scaler(NE_ST_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# define the temporal generalization object
time_gen = GeneralizingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
ne_ST_tgs = cross_val_multiscore(time_gen, ne_ST_X, ne_ST_y, cv=None, n_jobs=5)

# mean scores of decoding according to time point
ne_ST_tg = np.mean(ne_ST_tgs, axis=0)

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(ne_ST_tg, interpolation='lanczos', origin='lower',
               cmap='RdBu_r', extent=NE_ST_trial.times[[0, -1, 0, -1]],
               vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding performance : no-expectation block + second target')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
# train classifier on first target in no-expectation block

# make pipeline
clf_pipe = make_pipeline(Scaler(NE_C_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# define the temporal generalization object
time_gen = GeneralizingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
ne_C_tgs = cross_val_multiscore(time_gen, ne_C_X, ne_C_y, cv=None, n_jobs=5)

# mean scores of decoding according to time point
ne_C_tg = np.mean(ne_C_tgs, axis=0)

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(ne_C_tg, interpolation='lanczos', origin='lower',
               cmap='RdBu_r', extent=NE_C_trial.times[[0, -1, 0, -1]],
               vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding performance : no-expectation block + cue')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

### expectation block

In [ ]:
# train classifier on first target in expectation block

# make pipeline
clf_pipe = make_pipeline(Scaler(E_FT_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# define the temporal generalization object
time_gen = GeneralizingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_FT_tgs = cross_val_multiscore(time_gen, _, _, cv=None, n_jobs=5)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_FT_tg = np.mean(e_FT_tgs, axis=0)

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(e_FT_tg, interpolation='lanczos', origin='lower',
               cmap='RdBu_r', extent=E_FT_trial.times[[0, -1, 0, -1]],
               vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding performance : expectation block + first target')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
# train classifier on first target in expectation block

# make pipeline
clf_pipe = make_pipeline(Scaler(E_ST_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# define the temporal generalization object
time_gen = GeneralizingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_ST_tgs = cross_val_multiscore(time_gen, _, _, cv=None, n_jobs=5)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_ST_tg = np.mean(e_ST_tgs, axis=0)

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(e_ST_tg, interpolation='lanczos', origin='lower',
               cmap='RdBu_r', extent=E_ST_trial.times[[0, -1, 0, -1]],
               vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding performance : expectation block + second target')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
# train classifier on first target in expectation block

# make pipeline
clf_pipe = make_pipeline(Scaler(E_C_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# define the temporal generalization object
time_gen = GeneralizingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_C_tgs = cross_val_multiscore(time_gen, _, _, cv=None, n_jobs=5)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_C_tg = np.mean(e_C_tgs, axis=0)

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(e_C_tg, interpolation='lanczos', origin='lower',
               cmap='RdBu_r', extent=E_C_trial.times[[0, -1, 0, -1]],
               vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding performance : expectation block + cue')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

### expectation block + distractor-absent

In [ ]:
# train classifier on first target in distractor-absent trials

# make pipeline
clf_pipe = make_pipeline(Scaler(E_ND_FT_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# define the temporal generalization object
time_gen = GeneralizingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_nd_FT_tgs = cross_val_multiscore(time_gen, _, _, cv=None, n_jobs=5)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_nd_FT_tg = np.mean(e_nd_FT_tgs, axis=0)

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(e_nd_FT_tg, interpolation='lanczos', origin='lower',
               cmap='RdBu_r', extent=E_ND_FT_trial.times[[0, -1, 0, -1]],
               vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding performance : distractor-absent + first target')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
# train classifier on first target in distractor-absent trials

# make pipeline
clf_pipe = make_pipeline(Scaler(E_ND_ST_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# define the temporal generalization object
time_gen = GeneralizingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_nd_ST_tgs = cross_val_multiscore(time_gen, _, _, cv=None, n_jobs=5)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_nd_ST_tg = np.mean(e_nd_ST_tgs, axis=0)

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(e_nd_ST_tg, interpolation='lanczos', origin='lower',
               cmap='RdBu_r', extent=E_ND_ST_trial.times[[0, -1, 0, -1]],
               vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding performance : distractor-absent + second target')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
# train classifier on first target in distractor-absent trials

# make pipeline
clf_pipe = make_pipeline(Scaler(E_ND_C_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# define the temporal generalization object
time_gen = GeneralizingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_nd_C_tgs = cross_val_multiscore(time_gen, _, _, cv=None, n_jobs=5)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_nd_C_tg = np.mean(e_nd_C_tgs, axis=0)

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(e_nd_C_tg, interpolation='lanczos', origin='lower',
               cmap='RdBu_r', extent=E_ND_C_trial.times[[0, -1, 0, -1]],
               vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding performance : distractor-absent + cue')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

### expectation block + distractor-present

In [ ]:
# train classifier on first target in distractor-present trials

# make pipeline
clf_pipe = make_pipeline(Scaler(E_D_FT_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# define the temporal generalization object
time_gen = GeneralizingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_d_FT_tgs = cross_val_multiscore(time_gen, _, _, cv=None, n_jobs=5)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_d_FT_tg = np.mean(e_d_FT_tgs, axis=0)

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(e_d_FT_tg, interpolation='lanczos', origin='lower',
               cmap='RdBu_r', extent=E_D_FT_trial.times[[0, -1, 0, -1]],
               vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding performance : distractor-present + first target')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
# train classifier on first target in distractor-present trials

# make pipeline
clf_pipe = make_pipeline(Scaler(E_D_ST_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# define the temporal generalization object
time_gen = GeneralizingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_d_ST_tgs = cross_val_multiscore(time_gen, _, _, cv=None, n_jobs=5)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_d_ST_tg = np.mean(e_d_ST_tgs, axis=0)

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(e_d_ST_tg, interpolation='lanczos', origin='lower',
               cmap='RdBu_r', extent=E_D_ST_trial.times[[0, -1, 0, -1]],
               vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding performance : distractor-present + second target')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
# train classifier on first target in distractor-present trials

# make pipeline
clf_pipe = make_pipeline(Scaler(E_D_C_trial.info), Vectorizer(),
                         LogisticRegression(solver='liblinear'))

# define the temporal generalization object
time_gen = GeneralizingEstimator(clf_pipe, n_jobs=None, scoring='roc_auc', verbose=True)

# conduct cross validation
e_d_C_tgs = cross_val_multiscore(time_gen, _, _, cv=None, n_jobs=5)
# remove _, and enter correct variable name

# mean scores of decoding according to time point
e_d_C_tg = np.mean(e_d_C_tgs, axis=0)

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(e_d_C_tg, interpolation='lanczos', origin='lower',
               cmap='RdBu_r', extent=E_D_C_trial.times[[0, -1, 0, -1]],
               vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding performance : distractor-present + cue')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

## cross-condition decoding (within-block)

### no-expectation block

#### trained by first target

##### decode separately

In [ ]:
# train decoder based on first target and fit against second target

FT_pipe = make_pipeline(Scaler(NE_FT_trial.info), Vectorizer(),
                        LogisticRegression(solver='liblinear'))

print('Start decoding : First to Second')
ne_pred_FtoS = []
for i in range(len(ne_FT_X[0][0])):
    FT_pipe.fit(ne_FT_X[:,:,i], ne_FT_y)
    temp_pred_FtoS = np.array([2])
    for j in range(len(ne_ST_X[0][0])):
        pred = FT_pipe.predict(ne_ST_X[:,:,j])
        temp = np.array([2])
        for k in range(len(pred)):
            if (pred[k] == ne_ST_y[k]):
                temp = np.hstack((temp, 1))
            elif (pred[k] != ne_ST_y[k]):
                temp = np.hstack((temp, 0))
        temp = np.delete(temp, 0)
        temp_pred_FtoS = np.hstack((temp_pred_FtoS, np.mean(temp)))
    temp_pred_FtoS = np.delete(temp_pred_FtoS, 0)
    if (i == 0):
        ne_pred_FtoS = temp_pred_FtoS[None,:]
    else:
        ne_pred_FtoS = np.vstack((ne_pred_FtoS, temp_pred_FtoS[None,:]))
    if ((i+1)%10 == 0):
        print('progress : '+str(i+1)+'/'+str(len(ne_FT_X[0][0]))+'time points')
print('DONE!')

print('Start decoding : First to Cue')
ne_pred_FtoC = []
for i in range(len(ne_FT_X[0][0])):
    FT_pipe.fit(ne_FT_X[:,:,i], ne_FT_y)
    temp_pred_FtoC = np.array([2])
    for j in range(len(ne_C_X[0][0])):
        pred = FT_pipe.predict(ne_C_X[:,:,j])
        temp = np.array([2])
        for k in range(len(pred)):
            if (pred[k] == ne_C_y[k]):
                temp = np.hstack((temp, 1))
            elif (pred[k] != ne_C_y[k]):
                temp = np.hstack((temp, 0))
        temp = np.delete(temp, 0)
        temp_pred_FtoC = np.hstack((temp_pred_FtoC, np.mean(temp)))
    temp_pred_FtoC = np.delete(temp_pred_FtoC, 0)
    if (i == 0):
        ne_pred_FtoC = temp_pred_FtoC[None,:]
    else:
        ne_pred_FtoC = np.vstack((ne_pred_FtoC, temp_pred_FtoC[None,:]))
    if ((i+1)%10 == 0):
        print('progress : '+str(i+1)+'/'+str(len(ne_FT_X[0][0]))+'time points')
print('DONE!')

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(ne_pred_FtoS, interpolation='lanczos', origin='lower', cmap='RdBu_r', 
               extent=NE_ST_trial.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding reseult(NE) : First to Second')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(ne_pred_FtoC, interpolation='lanczos', origin='lower', cmap='RdBu_r', 
               extent=NE_C_trial.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding reseult(NE) : First to Second')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

##### decode at once

In [ ]:
# train decoder based on first target and predict second target and cue

FT_pipe = make_pipeline(Scaler(NE_FT_trial.info), Vectorizer(),
                        LogisticRegression(solver='liblinear'))

print('Start decoding : trained on First target')
ne_pred_FtoS = []
ne_pred_FtoC = []
for i in range(len(ne_FT_X[0][0])):
    FT_pipe.fit(ne_FT_X[:,:,i], ne_FT_y)
    temp_pred = np.array([2, 2])
    for j in range(len(ne_ST_X[0][0])):
        pred_S = FT_pipe.predict(ne_ST_X[:,:,j])
        pred_C = FT_pipe.predict(ne_C_X[:,:,j])
        temp = np.array([2, 2])
        t_temp = np.array([2, 2])[None,:]
        for k in range(len(pred_S)):
            if (pred_S[k] == ne_ST_y[k]):
                temp[0] = 1
            elif (pred_S[k] != ne_ST_y[k]):
                temp[0] = 0
                
            if (pred_C[k] == ne_C_y[k]):
                temp[1] = 1
            elif (pred_C[k] != ne_C_y[k]):
                temp[1] = 0
            t_temp = np.vstack((t_temp, temp))
        t_temp = np.delete(t_temp, 0, axis=0)
        temp_pred = np.vstack((temp_pred, np.mean(t_temp, axis=0)))
    temp_pred = np.delete(temp_pred, 0, axis=0)
    if (i == 0):
        ne_pred_FtoS = temp_pred[:,0][None,:]
        ne_pred_FtoC = temp_pred[:,1][None,:]
    else:
        ne_pred_FtoS = np.vstack((ne_pred_FtoS, temp_pred[:,0][None,:]))
        ne_pred_FtoC = np.vstack((ne_pred_FtoC, temp_pred[:,1][None,:]))
    if ((i+1)%10 == 0):
        print('progress : '+str(i+1)+'/'+str(len(ne_FT_X[0][0]))+'time points')
print('DONE!')

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(ne_pred_FtoS, interpolation='lanczos', origin='lower', cmap='RdBu_r', 
               extent=NE_ST_trial.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding reseult(NE) : First to Second')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(ne_pred_FtoC, interpolation='lanczos', origin='lower', cmap='RdBu_r', 
               extent=NE_C_trial.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding reseult(NE) : First to Second')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

##### make decoding function

In [ ]:
def cross_condition_dec(fit_trial, fit_X, fit_y, predict_X, predict_y, cond):
    
    predict_y = np.array(predict_y)
    train_pipe = make_pipeline(Scaler(fit_trial.info), Vectorizer(),
                               LogisticRegression(solver='liblinear'))

    print('Start decoding : ' + str(cond))
    decoding_result = []
    for i in range(len(fit_X[0][0])):
        train_pipe.fit(fit_X[:,:,i], fit_y)
        temp_result = np.array([2])
        for j in range(len(predict_X[0][0])):
            pred = train_pipe.predict(predict_X[:,:,j])
            temp = np.array([2])
            for k in range(len(pred)):
                if (pred[k] == predict_y[k]):
                    temp = np.hstack((temp, 1))
                elif (pred[k] != predict_y[k]):
                    temp = np.hstack((temp, 0))
            temp = np.delete(temp, 0)
            temp_result = np.hstack((temp_result, np.mean(temp)))
        temp_result = np.delete(temp_result, 0)
        if (i == 0):
            decoding_result = temp_result[None,:]
        else:
            decoding_result = np.vstack((decoding_result, temp_result[None,:]))
        if ((i+1)%10 == 0):
            print('progress : '+str(i+1)+'/'+str(len(fit_X[0][0]))+'trials')
    print('DONE!')
    
    return decoding_result

In [ ]:
ne_pred_FtoS = cross_condition_dec(NE_FT_trial, ne_FT_X, ne_FT_y,
                                   ne_ST_X, ne_ST_y, 'First to Second')

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(ne_pred_FtoS, interpolation='lanczos', origin='lower', cmap='RdBu_r', 
               extent=NE_ST_trial.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding reseult(NE) : First to Second')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
ne_pred_FtoC = cross_condition_dec(NE_FT_trial, ne_FT_X, ne_FT_y,
                                   ne_C_X, ne_C_y, 'First to Cue')

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(ne_pred_FtoC, interpolation='lanczos', origin='lower', cmap='RdBu_r', 
               extent=NE_C_trial.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding reseult(NE) : First to Cue')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

#### trained by second target

In [ ]:
ne_StoF = cross_condition_dec(NE_ST_trial, ne_ST_X, ne_ST_y,
                              ne_FT_X, ne_FT_y, 'Second to First')

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(ne_StoF, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=NE_FT_trial.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding reseult(NE) : Second to First')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
ne_pred_StoC = cross_condition_dec(NE_ST_trial, ne_ST_X, ne_ST_y,
                                   ne_C_X, ne_C_y, 'Second to Cue')

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(ne_pred_StoC, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=NE_C_trial.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding result(NE) : Second to Cue')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

#### trained by cue

In [ ]:
ne_pred_CtoF = cross_condition_dec(NE_C_trial, ne_C_X, ne_C_y,
                                   ne_FT_X, ne_FT_y, 'Cue to First')

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(ne_pred_CtoF, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=NE_FT_trial.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding reseult(NE) : Cue to First')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
ne_pred_CtoS = cross_condition_dec(NE_C_trial, ne_C_X, ne_C_y,
                                   ne_ST_X, ne_ST_y, 'Cue to Second')

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(ne_pred_CtoS, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=NE_ST_trial.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding result(NE) : Cue to Second')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

### expectation block

#### trained by first target

In [ ]:
e_pred_FtoS = cross_condition_dec(_, _, _,
                                  _, _, 'First to Second')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding reseult(NE) : First to Second')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
e_pred_FtoC = cross_condition_dec(_, _, _,
                                  _, _, 'First to Cue')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding result(E) : First to Cue')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

#### trained by second target

In [ ]:
e_pred_StoF = cross_condition_dec(_, _, _,
                                  _, _, 'Second to First')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding reseult(E) : Second to First')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
e_pred_StoC = cross_condition_dec(_, _, _,
                                  _, _, 'Second to Cue')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding result(E) : Second to Cue')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

#### trained by cue

In [ ]:
e_pred_CtoF = cross_condition_dec(_, _, _,
                                  _, _, 'Cue to First')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding result(E) : Cue to First')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
e_pred_CtoS = cross_condition_dec(_, _, _,
                                  _, _, 'Cue to Second')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding result(E) : Cue to Second')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

### expectation block + distractor-absent

In [ ]:
e_nd_pred_FtoS = cross_condition_dec(_, _, _,
                                     _, _, 'First to Second')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding reseult(NE) : First to Second')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
e_nd_pred_FtoC = cross_condition_dec(_, _, _,
                                     _, _, 'First to Cue')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding result(E) : First to Cue')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

#### trained by second target

In [ ]:
e_nd_pred_StoF = cross_condition_dec(_, _, _,
                                     _, _, 'Second to First')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding reseult(E) : Second to First')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
e_nd_pred_StoC = cross_condition_dec(_, _, _,
                                     _, _, 'Second to Cue')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding result(E) : Second to Cue')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

#### trained by cue

In [ ]:
e_nd_pred_CtoF = cross_condition_dec(_, _, _,
                                     _, _, 'Cue to First')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding result(E) : Cue to First')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
e_nd_pred_CtoS = cross_condition_dec(_, _, _,
                                     _, _, 'Cue to Second')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding result(E) : Cue to Second')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

### expectation block + distractor-present

In [ ]:
e_d_pred_FtoS = cross_condition_dec(_, _, _,
                                    _, _, 'First to Second')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding reseult(NE) : First to Second')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
e_d_pred_FtoC = cross_condition_dec(_, _, _,
                                    _, _, 'First to Cue')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding result(E) : First to Cue')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

#### trained by second target

In [ ]:
e_d_pred_StoF = cross_condition_dec(_, _, _,
                                    _, _, 'Second to First')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding reseult(E) : Second to First')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
e_d_pred_StoC = cross_condition_dec(_, _, _,
                                    _, _, 'Second to Cue')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding result(E) : Second to Cue')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

#### trained by cue

In [ ]:
e_d_pred_CtoF = cross_condition_dec(_, _, _,
                                    _, _, 'Cue to First')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding result(E) : Cue to First')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')

In [ ]:
e_d_pred_CtoS = cross_condition_dec(_, _, _,
                                    _, _, 'Cue to Second')
# remove _s, and enter correct variable names

In [ ]:
# plot data on 2-d space using same data with upper block

fig, ax = plt.subplots()
im = ax.imshow(_, interpolation='lanczos', origin='lower', cmap='RdBu_r',
               extent=_.times[[0, -1, 0, -1]], vmin=0., vmax=1.)
# remove _s, and enter correct variable names
ax.set_xlabel('Testing Time (s)')
ax.set_ylabel('Training Time (s)')
ax.set_title('decoding result(E) : Cue to Second')
ax.axvline(0, color='k')
ax.axhline(0, color='k')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('AUC')